In [1]:
# 이거 실행한 다음에 좌측 상단 '런타임 - 런타임 다시 시작' 을 클릭하여 런타임을 재시작해주세요! (plt 한글깨짐 방지)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-san

In [2]:
#나눔폰트사용
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

# 1번 csv
gangseo_parking = pd.read_csv('/content/drive/MyDrive/unite/data/강서구_거주자우선주차_구획정보.csv')
gangseo_parking.tail(2)

Mounted at /content/drive


,연번,구획라벨,소속동,소속구간,용도구분,공유종류,경도,위도
3424,3425,18-50-12,우장산동,18-50,일반,공유가능,126.8431866,37.55432428
3425,3426,18-45-101,우장산동,18-45,전용,공유가능,126.8405053,37.55256487


In [4]:
# 2번 csv
gangseo_illegal_parking = pd.read_csv('/content/drive/MyDrive/unite/data/강서구_거주자우선주차장_부정주차.csv')
gangseo_illegal_parking.tail(2)

,단속해당동,단속장소,차량번호,단속등록일
40908,가양1동,14_16,34****9,2021-11-30 오후 9:02
40909,화곡4동,9_81,25****65,2021-11-30 오후 9:10


In [5]:
gangseo_illegal_parking.rename(columns={'단속해당동':'소속동', '단속장소':'소속구간'}, inplace=True)

In [6]:
gangseo_illegal_parking = gangseo_illegal_parking[['소속동', '소속구간']]
gangseo_illegal_parking['소속구간'] = gangseo_illegal_parking['소속구간'].str.replace('_', '-')


<ipython-input-6-c8c80d57ef45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gangseo_illegal_parking['소속구간'] = gangseo_illegal_parking['소속구간'].str.replace('_', '-')


In [7]:
gangseo_parking = gangseo_parking[gangseo_parking.공유종류 != '공유불가능']

In [8]:
gangseo_parking = gangseo_parking[gangseo_parking.소속구간 != '헤링턴타워']

In [9]:
gangseo_parking['소속구간'].value_counts()

6-63     29
14-12    24
17-51    22
18-49    21
14-14    19
         ..
13-52     1
4-06      1
13-53     1
13-54     1
19-04     1
Name: 소속구간, Length: 513, dtype: int64

In [10]:
gangseo_parking = gangseo_parking[['소속구간', '경도', '위도']]

In [11]:
gangseo_illegal_parking['소속구간'].value_counts()

1-34         837
2-06         814
2-07         636
22-18        605
1-22         576
            ... 
19-29-101      1
20-01-10       1
3-15-06        1
19-25-01       1
19-71-02       1
Name: 소속구간, Length: 1358, dtype: int64

In [12]:
# gangseo_illegal_parking = gangseo_illegal_parking.groupby(['소속구간']).sum().reset_index()
gangseo_illegal_parking_count = gangseo_illegal_parking['소속구간'].value_counts().rename_axis('소속구간').reset_index(name='단속횟수')
gangseo_illegal_parking_count

,소속구간,단속횟수
0,1-34,837
1,2-06,814
2,2-07,636
3,22-18,605
4,1-22,576
...,...,...
1353,19-29-101,1
1354,20-01-10,1
1355,3-15-06,1
1356,19-25-01,1


In [13]:
# gangseo_illegal_parking_count['구획라벨'] = gangseo_illegal_parking_count['소속구간']
gangseo_illegal_parking

,소속동,소속구간
0,방화3동,22-23
1,화곡3동,8-06
2,화곡6동,11-61
3,화곡8동,13-19
4,화곡8동,13-19
...,...,...
40905,화곡1동,6-10
40906,가양1동,14-16
40907,가양1동,14-16
40908,가양1동,14-16


In [14]:
# gangseo_illegal_parking_count

data = {'소속구간' : []}
illegal_parking = pd.DataFrame(data)

for index, value in gangseo_illegal_parking[gangseo_illegal_parking['소속구간'].str.count(pat='-') == 2].iterrows():
  value = value.loc['소속구간']
  index_ = value.find('-', 3)
  value = value[:index_]
  illegal_parking.loc[index] = value

for index, value in gangseo_illegal_parking[gangseo_illegal_parking['소속구간'].str.count(pat='-') == 1].iterrows():
  value = value.loc['소속구간']
  illegal_parking.loc[index] = value

illegal_parking

,소속구간
10,6-18
13,6-27
41,7-24
63,13-17
66,6-18
...,...
40905,6-10
40906,14-16
40907,14-16
40908,14-16


In [15]:
illegal_parking_count = illegal_parking['소속구간'].value_counts().rename_axis('소속구간').reset_index(name='단속횟수')
illegal_parking_count

,소속구간,단속횟수
0,1-34,837
1,2-06,817
2,22-18,672
3,1-22,649
4,2-07,636
...,...,...
563,7-59,1
564,22-15,1
565,11-30,1
566,1-51,1


In [16]:
gangseo_parking

,소속구간,경도,위도
0,1-13,126.8743747,37.55293616
1,1-13,126.8743701,37.55298392
2,1-13,126.8743654,37.55303347
3,1-13,126.8743608,37.55308303
4,1-15,126.8762511,37.55051388
...,...,...,...
3421,18-50,126.8434072,37.55433584
3422,18-50,126.8433705,37.55430651
3423,18-50,126.8432488,37.55431536
3424,18-50,126.8431866,37.55432428


In [17]:
gangseo_parking = gangseo_parking[gangseo_parking.경도 != '값 없음']
gangseo_parking['경도'] = gangseo_parking['경도'].astype(float)
gangseo_parking['위도'] = gangseo_parking['위도'].astype(float)

<ipython-input-17-01f28645fed7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gangseo_parking['경도'] = gangseo_parking['경도'].astype(float)
<ipython-input-17-01f28645fed7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gangseo_parking['위도'] = gangseo_parking['위도'].astype(float)


In [18]:
df_test = gangseo_parking.groupby(['소속구간']).mean().reset_index()
df_test

,소속구간,경도,위도
0,1-13,126.874368,37.553009
1,1-15,126.876147,37.550540
2,1-20,126.877289,37.547999
3,1-22,126.876457,37.548297
4,1-34,126.869323,37.549674
...,...,...,...
506,9-82,126.860982,37.530575
507,9-83,126.861498,37.530733
508,9-84,126.862041,37.531091
509,9-86,126.863107,37.531056


In [19]:
df_merge = pd.merge(df_test, illegal_parking_count, how='left')
#df_merge

integrated_df = pd.concat([df_test, illegal_parking_count], axis=0)
integrated_df


,소속구간,경도,위도,단속횟수
0,1-13,126.874368,37.553009,NaN
1,1-15,126.876147,37.550540,NaN
2,1-20,126.877289,37.547999,NaN
3,1-22,126.876457,37.548297,NaN
4,1-34,126.869323,37.549674,NaN
...,...,...,...,...
563,7-59,NaN,NaN,1.0
564,22-15,NaN,NaN,1.0
565,11-30,NaN,NaN,1.0
566,1-51,NaN,NaN,1.0


In [20]:
a= df_merge[df_merge['소속구간'] == '7-59']
print(a)

     소속구간          경도         위도  단속횟수
413  7-59  126.853747  37.530529   1.0


In [21]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 511 entries, 0 to 510
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   소속구간    511 non-null    object 
 1   경도      511 non-null    float64
 2   위도      511 non-null    float64
 3   단속횟수    463 non-null    float64
dtypes: float64(3), object(1)
memory usage: 20.0+ KB


In [22]:
test = integrated_df[integrated_df['경도'].isnull() == True]
test

,소속구간,경도,위도,단속횟수
0,1-34,NaN,NaN,837.0
1,2-06,NaN,NaN,817.0
2,22-18,NaN,NaN,672.0
3,1-22,NaN,NaN,649.0
4,2-07,NaN,NaN,636.0
...,...,...,...,...
563,7-59,NaN,NaN,1.0
564,22-15,NaN,NaN,1.0
565,11-30,NaN,NaN,1.0
566,1-51,NaN,NaN,1.0


In [23]:
import folium
import math
import base64
from folium import IFrame

In [27]:
from IPython.utils.sysinfo import num_cpus
count = 0

gangseo_map = folium.Map(location=[37.5543,126.8434], zoom_start=15, scrollWheelZoom=False)
for lat, lng, block, num in zip(df_merge['위도'], df_merge['경도'], df_merge['소속구간'], df_merge['단속횟수']):
  try:
    folium.CircleMarker(
      [lat, lng],
      radius=math.sqrt(num),
      fill=True,
      color='black',
      fill_color='#0000ff99',
      popup =num,
      tooltip=block).add_to(gangseo_map)
  except:
    pass

In [28]:
# gangseo_map = folium.Map(location=[37.5543,126.8434], zoom_start=14, scrollWheelZoom=False)

for lat, lng in zip(gangseo_parking['위도'], gangseo_parking['경도']):
  iframe = IFrame(width=800, height=450)
  popup = folium.Popup(iframe, max_width=800)
  try:
    folium.CircleMarker(
      [lat, lng],
      radius=1,
      fill=True,
      color='red',
      fill_color='#0000ff99',
      popup ='here',
      tooltip='here').add_to(gangseo_map)
  except:
    pass

display(gangseo_map)
# seoul_illegal_parking

Output hidden; open in https://colab.research.google.com to view.